# Actividad 3: Comparando clasificadores
Integrantes del equipo:
- Francisco Arenas
- Víctor Curiel
- Víctor Rodríguez
- Alek Howland


In [10]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import svm
from sklearn.metrics import accuracy_score

#Leemos csv
data = pd.read_csv('iris.csv')
#Eliminamos la columna de Id
data = data.drop(columns=["Id"])

#Separamos en datos y etiquetas
X = data.iloc[:, :-1].to_numpy() #todas las filas, todas las columnas menos la última
Y = data.iloc[:, -1].to_numpy() #todas las filas, sólo la última columna

#Obtener datos de entrenamiento y de pruebas
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(X, Y, test_size=0.5) #test size es un %
#Del conjunto de pruebas obtener el conjunto de validación
xValidation, xTest, yValidation, yTest = model_selection.train_test_split(X, Y, test_size=0.5) 

#normalizar para SVM
normalizer = preprocessing.MinMaxScaler() #generar normalizador
normalizer = normalizer.fit(xTrain) #sólo hacer la normalización/estandarización con los datos de entrenamiento
xTrainNew = normalizer.transform(xTrain)

#transformar datos de validación y pruebas
xTestNew = normalizer.transform(xTest)
xValidationNew = normalizer.transform(xValidation)

# -ENTRENAMIENTO Y VALIDACIÓN-
print("FASE DE VALIDACION")
# KNN
#Probaremos con valores de k del 1 al 10 para determinar qué valor nos da un mayor accuracy
print("-KNN")
bestKValue = 0
bestAccKNN = 0
for k in range(1, 11): # valores de k para el proceso de validación del KNN
    knnClassifier = KNeighborsClassifier(n_neighbors=k)
    knnClassifier = knnClassifier.fit(xTrain, yTrain)
    acc = accuracy_score(yValidation, knnClassifier.predict(xValidation))
    if acc > bestAccKNN:
        bestKValue = k
        bestAccKNN = acc
    print("Con k = " + str(k) + " tenemos un accuracy de " + str(acc))
print("Con k = " + str(bestKValue) + " obtenemos el mejor accuracy")

# SVM
#Probaremos con diferentes kernels para determinar qué valor nos da un mayor accuracy
print("-SVM")
kernels = ["linear", "rbf", "poly"]
bestSVMKernel = -1
bestAccSVM = 0
for k in kernels: # valores de k para el proceso de validación del KNN
    svmClassifier = svm.SVC(kernel=k)
    svmClassifier = svmClassifier.fit(xTrainNew, yTrain)
    acc = accuracy_score(yValidation, svmClassifier.predict(xValidationNew))
    if acc > bestAccSVM:
        bestAccSVM = acc
        bestSVMKernel = kernels.index(k)
    print("Con el kernel " + str(k) + " tenemos un accuracy de " + str(acc))
print("El kernel con el que obtenemos un mayor accuracy es " + kernels[bestSVMKernel])

# Árboles de decisión - J48
dTreeClassifier = DecisionTreeClassifier()
dTreeClassifier = dTreeClassifier.fit(xTrain,yTrain)

print("FASE DE PRUEBAS")
print("Con accuracy:")
knnClassifier = KNeighborsClassifier(n_neighbors=bestKValue)
knnClassifier = knnClassifier.fit(xTrain, yTrain)
print("KNN: " + str(accuracy_score(yTest, knnClassifier.predict(xTest))))
print("J48: " + str(accuracy_score(yTest, dTreeClassifier.predict(xTest))))
svmClassifier = svm.SVC(kernel=kernels[bestSVMKernel])
svmClassifier = svmClassifier.fit(xTrainNew, yTrain)
print("SVM: " + str(accuracy_score(yTest, svmClassifier.predict(xTestNew))))
print("Con Cross-Validation (3 folds):")
print("KNN: ", end="")
print(model_selection.cross_val_score(knnClassifier, xTest, yTest, cv=3))
print("J48: ", end="")
print(model_selection.cross_val_score(dTreeClassifier, xTest, yTest, cv=3))
print("SVM: ", end="")
print(model_selection.cross_val_score(svmClassifier, xTestNew, yTest, cv=3))


FASE DE VALIDACION
-KNN
Con k = 1 tenemos un accuracy de 0.9866666666666667
Con k = 2 tenemos un accuracy de 0.96
Con k = 3 tenemos un accuracy de 0.9866666666666667
Con k = 4 tenemos un accuracy de 0.96
Con k = 5 tenemos un accuracy de 0.9866666666666667
Con k = 6 tenemos un accuracy de 0.9866666666666667
Con k = 7 tenemos un accuracy de 0.9866666666666667
Con k = 8 tenemos un accuracy de 0.9733333333333334
Con k = 9 tenemos un accuracy de 0.9733333333333334
Con k = 10 tenemos un accuracy de 0.96
Con k = 1 obtenemos el mejor accuracy
-SVM
Con el kernel linear tenemos un accuracy de 0.96
Con el kernel rbf tenemos un accuracy de 0.96
Con el kernel poly tenemos un accuracy de 0.9866666666666667
El kernel con el que obtenemos un mayor accuracy es poly
FASE DE PRUEBAS
Con accuracy:
KNN: 0.9733333333333334
J48: 0.9733333333333334
SVM: 0.9733333333333334
Con Cross-Validation (3 folds):
KNN: [0.88 0.96 1.  ]
J48: [0.92 0.92 0.92]
SVM: [0.92 0.92 0.84]
